In [1]:
from datasets import load_dataset, Dataset, Audio
from transformers import AutoProcessor, WhisperModel, AutoTokenizer, AutoModelForCausalLM
import torch
import wave

/home/dhuser/Desktop/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-31 11:23:00.250469: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-31 11:23:00.283289: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-31 11:23:00.847494: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Load the Audio Data
list_of_audio_files = ["data/sub/De95Osq7p1c_trimmed_segment_1.wav", "data/sub/De95Osq7p1c_trimmed_segment_2.wav"]

In [103]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor

model_generate = WhisperForConditionalGeneration.from_pretrained("openai/whisper-medium")
model_generate.to(device)

generated = model_generate.generate(inputs = inputs)
processor.batch_decode(generated)

/home/dhuser/Desktop/.venv/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:473: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


['<|startoftranscript|><|th|><|transcribe|><|notimestamps|>ทุกคน วันนี้น่ะมีเด็กกับผู้ชายคนหนึ่งค่ะ<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>',
 '<|startoftranscript|><|th|><|transcribe|><|notimestamps|>หนุ่มลูกครึ่ง ชื่อแบบตาโตเซอร์ ๆ ผมยาวค้อ ๆ<|endoftext|>']

# Experimenting Adaptors

In [3]:
class AudioToTextPipeline(torch.nn.Module):
    def __init__(self, audio_encoder="openai/whisper-medium", llm="sealion"):
        super().__init__()
        
        # Device
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Load the Whisper model and processor
        self.audio_processor = AutoProcessor.from_pretrained(audio_encoder)
        self.audio_encoder = WhisperModel.from_pretrained(audio_encoder).to(self.device)
        
        # Freeze Audio Encoder weights
        for param in self.audio_encoder.parameters():
            param.requires_grad = False

        # Define the Adaptor
        self.adaptor = torch.nn.Linear(1024, 4096)  # Do we need bias?
        
        # Load the LLM and its tokenizer
        self.llm = llm
        
        # Freeze LLM weights

    def forward(self, audio_file_path):
        pass

    def process_and_encode_audio(self, list_audio_filepaths):
        def prepare_dataset(batch):
            audio = batch["audio"]
            batch["input_features"] = self.audio_processor.feature_extractor(audio["array"], 
                                                            sampling_rate=audio["sampling_rate"],
                                                            return_tensors='pt')['input_features'][0]
            return batch
        
        audio_dataset = Dataset.from_dict({
                "audio": list_audio_filepaths
            }
        )
        audio_dataset = audio_dataset.cast_column("audio", Audio())

        # Maps the audio files into Huggingface Dataset Format
        audio_dataset = audio_dataset.map(prepare_dataset)
        inputs = torch.tensor(audio_dataset['input_features']).to(self.device)

        # Ensuring No Gradient Updates during Encoding
        with torch.no_grad():
            encoder_outputs = self.audio_encoder.encoder(inputs, output_hidden_states=True)
        
        return encoder_outputs.last_hidden_state


In [4]:
translator = AudioToTextPipeline()

/home/dhuser/Desktop/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
translator.process_and_encode_audio(list_of_audio_files).shape

Map: 100%|██████████| 2/2 [00:00<00:00, 19.90 examples/s]


torch.Size([2, 1500, 1024])